# Real Time Notebook

## Spectrum analyser

In [ ]:
import struct
import wave

import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import pyaudio

# next line for VS Code
%matplotlib widget 
# next line for jupyter notebook
#%matplotlib notebook 

TITLE = ''
WIDTH = 640
HEIGHT = 480
FPS = 25

nFFT = 512
BUF_SIZE = 2 * nFFT
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

def animate(i, line, stream, MAX_y):
    N = int(stream.get_read_available())
    if N>0:
        N = int(max(N / nFFT, 1) * nFFT)
        frames = []  
        data = stream.read(N)
        frames.append(data)
        X=np.frombuffer(b''.join(frames),dtype=np.int16)/MAX_y
        Y = 20*np.log10(np.abs(np.fft.rfft(X, nFFT))+1.0e-4)
        line.set_ydata(Y)
    return line,

#
dpi = plt.rcParams['figure.dpi']
plt.rcParams['savefig.dpi'] = dpi
plt.rcParams["figure.figsize"] = (1.0 * WIDTH / dpi, 1.0 * HEIGHT / dpi)

fig = plt.figure()

X=np.zeros(nFFT)
X[:3]=1
Y = 20*np.log10(np.abs(np.fft.rfft(X, nFFT))/3)

# Frequency range
x_f = 1.0 * np.arange(nFFT/2+1) / nFFT * RATE

ax = fig.add_subplot(111, title=TITLE, xlim=(x_f[0], x_f[-1]), ylim=(-30,40))
line, = ax.plot(x_f, Y)

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                  channels=CHANNELS,
                  rate=RATE,
                  input=True,
                  frames_per_buffer=BUF_SIZE)

MAX_y = 2.0 ** (p.get_sample_size(FORMAT) * 8 - 1)
ani = animation.FuncAnimation(fig, animate, fargs=(line, stream, MAX_y), interval=1000.0 / FPS)

plt.show()


In [2]:
stream.stop_stream()
stream.close()
p.terminate()

NameError: name 'stream' is not defined